In [1]:
from gensim.models import KeyedVectors
import csv
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import igraph
from gensim.test.utils import datapath

### Get The embeddings

In [3]:
wv_from_text_PMC = KeyedVectors.load_word2vec_format("../../../../../data/PMC/wikipedia-pubmed-and-PMC-w2v.bin", binary=True)
wv_from_text_GV  = KeyedVectors.load_word2vec_format("../../../../../data/Glove_Gigaword/model.txt", binary=False)
wv_from_text_GM  = KeyedVectors.load_word2vec_format("../../../../../data/Gensim_wiki/model.txt", binary=False)
wv_from_text_FT  = KeyedVectors.load_word2vec_format("../../../../../data/FastText_wiki/model.txt", binary=False)

In [4]:
nb_top_terms=20

### Graph with NPMI matrix

In [20]:
f="../../../../../data/PubMed5/NPMI/allNPMI_30w_k3_otitis.csv"
NPMI=pd.read_csv(f)
NPMI=NPMI.set_index(['Unnamed: 0'])
NPMI

,otitis,infection,pneumonia,bacterial,acute,chronic,antibiotics,effusion,recurrent,complication,...,tympan,found,ag,present,mastoiditis,aom,children,year,dai,om
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
otitis,0.00000,0.48701,-0.28542,0.44146,0.22219,0.47456,0.39581,-0.06334,0.26187,-0.20776,...,0.43035,0.38669,0.36467,-0.34398,-0.00299,0.03365,-0.01772,-0.22582,-0.08150,-0.19186
infection,0.48701,0.00000,0.02468,0.01163,0.18929,0.14070,0.16808,-0.22721,0.15994,-0.37163,...,0.16180,0.09773,0.12970,-0.07480,-0.16686,-0.13022,-0.18159,0.09597,-0.24537,-0.35573
pneumonia,-0.28542,0.02468,0.00000,0.02054,-0.04180,0.01464,-0.37793,-0.26232,-0.05683,-0.40675,...,-0.37423,-0.39834,-0.40379,0.03622,-0.20197,-0.16534,-0.21671,-0.42480,-0.28048,-0.39084
bacterial,0.44146,0.01163,0.02054,0.00000,-0.00231,0.02749,-0.40251,-0.28690,-0.07883,-0.02664,...,0.03885,0.01338,-0.02939,0.04688,-0.22655,-0.18992,-0.24129,-0.04382,-0.30506,-0.41542
acute,0.22219,0.18929,-0.04180,-0.00231,0.00000,-0.02520,-0.41891,-0.30330,0.02159,-0.44773,...,0.00253,-0.43932,-0.03925,-0.01181,-0.24296,-0.20632,-0.25769,-0.05986,-0.32147,-0.43183
chronic,0.47456,0.14070,0.01464,0.02749,-0.02520,0.00000,0.24369,-0.20617,0.04605,-0.35060,...,0.27042,0.33006,0.29269,-0.07677,-0.14582,-0.10919,-0.16056,0.28213,-0.22433,0.22446
antibiotics,0.39581,0.16808,-0.37793,-0.40251,-0.41891,0.24369,0.00000,-0.15585,-0.35872,-0.30027,...,-0.26776,0.31480,0.46206,-0.43649,-0.09550,-0.05886,-0.11023,0.23454,-0.17401,-0.28437
effusion,-0.06334,-0.22721,-0.26232,-0.28690,-0.30330,-0.20617,-0.15585,0.00000,-0.24311,-0.18467,...,-0.15215,-0.17626,-0.18171,-0.32089,0.02011,0.05674,0.00537,-0.20272,-0.05840,-0.16876
recurrent,0.26187,0.15994,-0.05683,-0.07883,0.02159,0.04605,-0.35872,-0.24311,0.00000,-0.38753,...,-0.35502,0.06769,-0.38458,-0.10841,-0.18276,-0.14612,-0.19750,0.08892,-0.26127,-0.37163


In [21]:
NPMI=NPMI.iloc[0:nb_top_terms,0:nb_top_terms]
NPMI

,otitis,infection,pneumonia,bacterial,acute,chronic,antibiotics,effusion,recurrent,complication,ear,influenza,pneumococcal,cause,media,hear,isolate,middle,tube,vaccin
Unnamed: 0,,,,,,,,,,,,,,,,,,,,
otitis,0.00000,0.48701,-0.28542,0.44146,0.22219,0.47456,0.39581,-0.06334,0.26187,-0.20776,-0.22570,0.21981,0.55789,-0.41047,0.43285,-0.19659,0.44025,-0.35600,0.19867,-0.25862
infection,0.48701,0.00000,0.02468,0.01163,0.18929,0.14070,0.16808,-0.22721,0.15994,-0.37163,-0.38957,0.28045,0.27857,-0.07288,0.14717,-0.36046,0.30489,0.01146,0.09447,0.03046
pneumonia,-0.28542,0.02468,0.00000,0.02054,-0.04180,0.01464,-0.37793,-0.26232,-0.05683,-0.40675,-0.42468,-0.00602,-0.31280,0.01836,-0.02271,-0.39558,-0.04899,0.01973,-0.04627,-0.45761
bacterial,0.44146,0.01163,0.02054,0.00000,-0.00231,0.02749,-0.40251,-0.28690,-0.07883,-0.02664,0.10234,-0.01591,0.15831,-0.04083,-0.00034,-0.42016,-0.01170,-0.01593,-0.04009,-0.09737
acute,0.22219,0.18929,-0.04180,-0.00231,0.00000,-0.02520,-0.41891,-0.30330,0.02159,-0.44773,-0.06308,-0.02461,0.12738,-0.01163,-0.02057,-0.43656,-0.00721,-0.01417,-0.03995,0.01517
chronic,0.47456,0.14070,0.01464,0.02749,-0.02520,0.00000,0.24369,-0.20617,0.04605,-0.35060,-0.36854,0.09826,-0.25665,-0.05593,0.50645,-0.33943,0.36733,0.04811,0.20786,0.08821
antibiotics,0.39581,0.16808,-0.37793,-0.40251,-0.41891,0.24369,0.00000,-0.15585,-0.35872,-0.30027,-0.31821,-0.40248,-0.20632,-0.06762,0.23682,-0.28910,0.45937,0.00486,0.16889,0.19103
effusion,-0.06334,-0.22721,-0.26232,-0.28690,-0.30330,-0.20617,-0.15585,0.00000,-0.24311,-0.18467,-0.20260,-0.28687,-0.09071,-0.38738,-0.20632,-0.17349,-0.22907,-0.33290,-0.32099,-0.23553
recurrent,0.26187,0.15994,-0.05683,-0.07883,0.02159,0.04605,-0.35872,-0.24311,0.00000,-0.38753,-0.40547,-0.01582,-0.29358,-0.10440,0.02685,-0.37636,0.08327,-0.04112,0.06649,0.04461


In [17]:
#il faudra crée un fichier avec la frequence des tops termes pour les noeuds 
def graphNPMI(npmimatrix,pathfile, color):
    #Topterms 
    vect=npmimatrix.index
    topterms=['']*len(vect)
    for i in range(len(vect)):
        topterms[i]=vect[i]
    
    #frequences
    npmi=[0]*len(topterms)
    i=0
    with open(pathfile) as csvDataFile:
        csvReader = csv.reader(csvDataFile)
        for row in csvReader:
            if not(len(row)==0):
                #print(row[0])
                for v in topterms :
                    if(row[0] in v)and(row[0][0]==v[0]):
                        npmi[i]=float(row[1])
                        #print(frequences[i])
                        i=i+1
                        break

    # Make a graph from the adjacency matrix
    g = igraph.Graph.Weighted_Adjacency((npmimatrix.values > 0).tolist(), mode = "undirected")

    # Set words as vertices label
    g.vs["name"] = topterms
    g.vs["label"] = g.vs["name"]
    g.vs["color"] = color

    # Set size as vertices size by frequence of term in documents
    size_vertex =npmi 
    #normalize the vectors
    #size_vertex_norm = [float(i)/(sum(size_vertex) - size_vertex[0])*800  for i in size_vertex]
    g.vs["size"] = size_vertex

    # Set cosine similarity as edge width
    edge_w = [0]*len(g.get_edgelist())
    
    for ie, (i,j) in enumerate(g.get_edgelist()):
        if i == j:
            edge_w[ie] = 0
        else:
            edge_w[ie] = (npmimatrix.iloc[i][j] + 2)
    edge_w_color = [""]*len(g.get_edgelist())
    #il faut trouver un moyen de trouver les intervalles automatiquement
    for ie, edge_value in enumerate(edge_w):
        if (edge_value > 2.4):
            edge_w_color[ie] = "#126600" 
        elif (edge_value > 2.35 and edge_value < 2.4):
            edge_w_color[ie] = "#25cc00"
        else:
            edge_w_color[ie] = "#2eff00"

    g.es["weight"] = edge_w
    g.es["width"] = g.es["weight"]
    g.es["color"] = edge_w_color

    return g

In [18]:
# Variables correspondant aux couleurs des noeuds
# A modifier si besoin
colorNodeEmbeddingBasic = '#fbff28' #jaune
colorNodePMC = '#ff0000' #rouge
colorNodeGlove = '#287eff' #bleue
colorNodeFastText = '#d000f9' #violet
colorNodeGensim = '#f2b204' #orange

In [22]:
#il va falloir enregistrer le mot est sa frequence pour verifier si le mot n'a pas était viré
g = graphNPMI(NPMI,'../../../../../data/Palmetto/Npmi_for_graphs/npmi_otitis.csv', colorNodeEmbeddingBasic)
# plot the graph
igraph.summary(g)
out = igraph.plot(g, margin= 50, bbox = (700,700))
out.save('../../../../../output_Graph/quatriemeversion/Graph_All_embeddings_otitis.png')

IGRAPH UNW- 20 66 -- 
+ attr: color (v), label (v), name (v), size (v), color (e), weight (e), width (e)
